In [1]:
using Pkg
Pkg.activate(joinpath(Pkg.devdir(),"WebotsSim"));

In [3]:
using LightGraphs, MetaGraphs
using DataStructures
using JuMP, MathOptInterface
using Gurobi
using TOML
using Random

using GraphUtils
using TaskGraphs
using CRCBS
using WebotsSim

In [4]:
include(joinpath(dirname(pathof(WebotsSim)),"..","test","notebooks","render_tools.jl"));

┌ Info: Recompiling stale cache file /home/kylebrown/.julia/compiled/v1.0/Fontconfig/k95Ux.ji for Fontconfig [186bb1d3-e1f7-5a2c-a377-96d770f13627]
└ @ Base loading.jl:1190
ERROR: LoadError: Fontconfig.jl not properly installed. Please run `Pkg.build("Fontconfig")` and restart Julia.
Stacktrace:
 [1] error(::String) at ./error.jl:33
 [2] top-level scope at /home/kylebrown/.julia/packages/Fontconfig/EPLI0/src/Fontconfig.jl:7
 [3] include at ./boot.jl:317 [inlined]
 [4] include_relative(::Module, ::String) at ./loading.jl:1044
 [5] include(::Module, ::String) at ./sysimg.jl:29
 [6] top-level scope at none:2
 [7] eval at ./boot.jl:319 [inlined]
 [8] eval(::Expr) at ./client.jl:393
 [9] top-level scope at ./none:3
in expression starting at /home/kylebrown/.julia/packages/Fontconfig/EPLI0/src/Fontconfig.jl:4


LoadError: Failed to precompile Fontconfig [186bb1d3-e1f7-5a2c-a377-96d770f13627] to /home/kylebrown/.julia/compiled/v1.0/Fontconfig/k95Ux.ji.

# Toy Problem

In [ ]:
project_spec, problem_spec, robot_ICs, assignments, env_graph = pctapf_problem_3(;verbose=false);

env = construct_search_env(project_spec, problem_spec, robot_ICs, assignments, env_graph);
pc_mapf = PC_MAPF(env);
node = initialize_root_node(pc_mapf);
solver = PC_TAPF_Solver();

In [ ]:
plot_graph_bfs(project_spec.graph)

In [ ]:
plot_graph_bfs(problem_spec.graph)

In [ ]:
rg = get_display_metagraph(env.schedule;f=(v,p)->string(v,",",get_path_spec(env.schedule,v).path_id,",",get_path_spec(env.schedule,v).agent_id))
# rg = get_display_metagraph(env.schedule;f=(v,p)->string(env.cache.t0[v]," - ",env.cache.tF[v],
#         "\n",env.cache.local_slack[v]," - ",env.cache.slack[v]))
plot_graph_bfs(rg)

# Generate a random problem instance

In [ ]:
N = 8
M = 16
i = 2
# experiment_dir = joinpath(dirname(pathof(WebotsSim)),"..","experiments")

filename = string(WebotsSim.ENVIRONMENT_DIR,"/env_",i,".toml");
factory_env = read_env(filename);

Random.seed!(0);
project_spec, problem_spec, robot_ICs, env_graph = initialize_test_problem(N,M);

r0 = [get_id(get_location_id(robot_ICs[k])) for k in 1:problem_spec.N];
s0 = [get_id(get_location_id(project_spec.initial_conditions[k])) for k in 1:problem_spec.M];

In [ ]:
# Task Assignment
model = formulate_optimization_problem(problem_spec,Gurobi.Optimizer;OutputFlag=0);
optimize!(model);
optimal = (termination_status(model) == MathOptInterface.OPTIMAL);
@show optimal;
assignment_matrix = get_assignment_matrix(model);
assignments = map(j->findfirst(assignment_matrix[:,j] .== 1),1:M);
optimal_TA_cost = maximum(Int.(round.(value.(model[:tof])))); # lower bound on cost (from task assignment module)

In [ ]:
# Routing
project_schedule = construct_project_schedule(project_spec, problem_spec, robot_ICs, assignments);
env = construct_search_env(project_spec, problem_spec, robot_ICs, assignments, env_graph);
pc_mapf = PC_MAPF(env);
node = initialize_root_node(pc_mapf);
solver = PC_TAPF_Solver(verbosity=1);

In [ ]:
# for i in 1:get_num_vtxs(env.schedule)+1
#     plan_next_path!(solver,env,mapf,node);
# end
# paths1 = convert_to_vertex_lists(node.solution)
# reset_cache!(env.cache, env.schedule)
# for i in 1:get_num_vtxs(env.schedule)+1
#     plan_next_path!(solver,env,mapf,node);
# end
# paths2 = convert_to_vertex_lists(node.solution)

# tf = maximum(map(p->length(p),paths1))
# # set_default_plot_size(15cm,15cm)
# # record_video("paths1.webm",t->render_paths(t,paths1))
# # record_video("paths2.webm",t->render_paths(t,paths2))
# # set_default_plot_size(30cm,15cm)
# record_video("side_by_side.webm",t->render_both(t,paths1,paths2);tf=tf,s=(8inch,4inch))

# Solve a large problem instance with the full algorithm

In [ ]:
N = 20
M = 30
Random.seed!(1);
project_spec, problem_spec, robot_ICs, env_graph = initialize_test_problem(N,M);
solver = PC_TAPF_Solver(verbosity=0,max_A_star_iterations=10*nv(env_graph));
solution, cost = high_level_search!(solver, env_graph, project_spec, problem_spec, robot_ICs, Gurobi.Optimizer);
conflict_table = detect_conflicts(get_paths(solution));
@test !CRCBS.is_valid(get_next_conflict(conflict_table))

In [ ]:
conflict_table = detect_conflicts(get_paths(solution))
@show CRCBS.is_valid(get_next_conflict(conflict_table));

In [ ]:
r0 = [get_id(get_location_id(robot_ICs[k])) for k in 1:problem_spec.N];
s0 = [get_id(get_location_id(project_spec.initial_conditions[k])) for k in 1:problem_spec.M];

In [ ]:
paths = convert_to_vertex_lists(solution)
tf = maximum(map(p->length(p),paths))
set_default_plot_size(15cm,15cm)
record_video("large_problem_paths.webm",t->render_paths(t,paths);tf=tf,s=(8inch,8inch))